# Abschluss

- Line bzw Barchart Master vs Bachelor u4ber die Jahre aggregiert für FB
- Treemap für alle Studiengänge

In [6]:
import plotly.io as pio

# Benutzerdefiniertes Template definieren
infoviz_template = dict(
    layout=dict(
        template="plotly_white",
        title=dict(
            font=dict(size=20, family="Arial", weight="bold", color="black"),
            y=0.91,  # Titel weiter nach oben
            x=0.05,  # Links ausgerichtet
            xanchor="left",  # Linksbündig
        ),
        xaxis=dict(
            showgrid=False,
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelposition="outside bottom"
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey",
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelposition="outside left"
        ),
    )
)
pio.templates["infoviz"] = infoviz_template

In [1]:
import pandas as pd

# Datei einlesen (ersetze 'data.xlsx' durch den tatsächlichen Dateinamen)
file_path = "data.xlsx"

# Alle Sheets einlesen, aber das erste Blatt ignorieren
sheets = pd.read_excel(file_path, sheet_name=None, engine="openpyxl")

# Erstes Blatt entfernen
sheets.pop(next(iter(sheets)))

# Container für die bereinigten DataFrames
cleaned_sheets = {}

# Durch alle Jahre iterieren und DataFrames bereinigen
for jahr, df in sheets.items():
    df.columns = df.columns.astype(str).str.strip()  # Spaltennamen bereinigen
    cleaned_sheets[jahr] = df  # Speichern der bereinigten Daten

In [90]:
abschluss_anteile = {}

for jahr, df in cleaned_sheets.items():
    # Spalten automatisch nach Bachelor und Master aufteilen
    bachelor_columns = [col for col in df.columns if "Bachelor" in col]
    master_columns = [col for col in df.columns if "Master" in col]

    # Extrahiere die Gesamtanzahl der Studierenden pro Abschluss
    total_bachelor = df.loc[df["Category"] == "Anzahl Studierende", bachelor_columns].sum().sum()
    total_master = df.loc[df["Category"] == "Anzahl Studierende", master_columns].sum().sum()

    total_ruecklauf_bachelor = df.loc[df["Category"] == "Anzahl Teilnahmen an Studierendenbefragung", bachelor_columns].sum().sum()
    total_ruecklauf_master = df.loc[df["Category"] == "Anzahl Teilnahmen an Studierendenbefragung", master_columns].sum().sum()

    # Prozentanteile berechnen
    total_students = total_bachelor + total_master
    anteil_bachelor = (total_bachelor / total_students * 100) if total_students > 0 else 0
    anteil_master = (total_master / total_students * 100) if total_students > 0 else 0

    # Gesamtzahlen berechnen
    gesamt_ruecklauf = total_ruecklauf_bachelor + total_ruecklauf_master
    gesamt_ruecklaufquote = gesamt_ruecklauf / total_students * 100
    nicht_teilgenommen = total_students - gesamt_ruecklauf
    

    # Ergebnisse speichern
    abschluss_anteile[jahr] = {
        "Absolute Anzahl Bachelor": total_bachelor,
        "Absolute Anzahl Master": total_master,
        "Absoluter Rücklauf Bachelor": total_ruecklauf_bachelor,
        "Absoluter Rücklauf Master": total_ruecklauf_master,
        "Gesamtzahl Studierende": total_bachelor + total_master,
        "Gesamter Rücklauf": gesamt_ruecklauf,
        "Gesamte Rücklaufquote": gesamt_ruecklaufquote,
        "Gesamt nicht teilgenommen": nicht_teilgenommen,
        "Anteil Bachelor (%)": anteil_bachelor,
        "Anteil Master (%)": anteil_master,
    }

# DataFrame erstellen
abschluss_df = pd.DataFrame.from_dict(abschluss_anteile, orient="index").reset_index().rename(columns={"index": "Jahr"})

abschluss_df.head()  # Zur Kontrolle anzeigen

,Jahr,Absolute Anzahl Bachelor,Absolute Anzahl Master,Absoluter Rücklauf Bachelor,Absoluter Rücklauf Master,Gesamtzahl Studierende,Gesamter Rücklauf,Gesamte Rücklaufquote,Gesamt nicht teilgenommen,Anteil Bachelor (%),Anteil Master (%)
0,2013,1406.0,801.0,409.0,231.0,2207.0,640.0,28.998641,1567.0,63.706389,36.293611
1,2014,1580.0,767.0,453.0,241.0,2347.0,694.0,29.569663,1653.0,67.319983,32.680017
2,2015,1710.0,747.0,433.0,198.0,2457.0,631.0,25.681726,1826.0,69.597070,30.402930
3,2016,1795.0,967.0,494.0,205.0,2762.0,699.0,25.307748,2063.0,64.989138,35.010862
4,2017,1848.0,1236.0,491.0,267.0,3084.0,758.0,24.578470,2326.0,59.922179,40.077821


In [92]:
import plotly.express as px

fig = px.line(
    abschluss_df,
    x="Jahr",
    y="Gesamte Rücklaufquote",
    markers=True,
    title="Entwicklung der Abschlussarten (Bachelor vs. Master)",
    color_discrete_sequence=px.colors.qualitative.Safe,
    template="infoviz"
)

fig.update_traces(line=dict(width=3))  # Linie dicker machen

fig.update_layout(
    yaxis=dict(range=[0, 100]),  # Prozentwerte zwischen 0 und 100
)

fig.show()

In [89]:
import plotly.express as px

fig = px.bar(
    abschluss_df,
    x="Jahr",
    y=["Gesamter Rücklauf", "Gesamt nicht teilgenommen" ],
    title="Entwicklung Facherbeichsgröße und Teilnahemanzahl",
    color_discrete_sequence=px.colors.qualitative.Safe,
    template="infoviz"
)


# fig.update_layout(
#     yaxis=dict(range=[0, 600]),  # Prozentwerte zwischen 0 und 100
# )

fig.show()

In [82]:
import plotly.express as px

fig = px.line(
    abschluss_df,
    x="Jahr",
    y=["Absoluter Rücklauf Bachelor", "Absoluter Rücklauf Master"],
    markers=True,
    title="Entwicklung der Abschlussarten (Bachelor vs. Master)",
    color_discrete_sequence=px.colors.qualitative.Safe,
    template="infoviz"
)

fig.update_traces(line=dict(width=3))  # Linie dicker machen

fig.update_layout(
    yaxis=dict(range=[0, 600]),  # Prozentwerte zwischen 0 und 100
)

fig.show()

In [79]:
import plotly.express as px

fig = px.line(
    abschluss_df,
    x="Jahr",
    y=["Absolute Anzahl Bachelor", "Absolute Anzahl Master"],
    markers=True,
    title="Entwicklung der Abschlussarten (Bachelor vs. Master)",
    color_discrete_sequence=px.colors.qualitative.Safe,
    template="infoviz"
)

fig.update_traces(line=dict(width=3))  # Linie dicker machen

fig.update_layout(
    yaxis=dict(range=[0, 2100]),  # Prozentwerte zwischen 0 und 100
)

fig.show()

In [7]:
import plotly.express as px

# DataFrame für Plot umstrukturieren
abschluss_df_melted = abschluss_df.melt(id_vars="Jahr", var_name="Abschluss", value_name="Anteil (%)")

fig = px.line(
    abschluss_df_melted,
    x="Jahr",
    y="Anteil (%)",
    color="Abschluss",
    markers=True,
    title="Entwicklung der Abschlussarten (Bachelor vs. Master)",
    color_discrete_sequence=px.colors.qualitative.Safe,
    template="infoviz"
)

fig.update_traces(line=dict(width=3))  # Linie dicker machen

fig.update_layout(
    yaxis=dict(range=[0, 100]),  # Prozentwerte zwischen 0 und 100
    xaxis=dict(range=[2013, 2025])
)

fig.show()

In [34]:
import plotly.express as px


df_2024 = cleaned_sheets["2024"]

# Studiengänge automatisch erkennen (Bachelor und Master)
studiengaenge = [col for col in df_2024.columns if "Bachelor" in col or "Master" in col]

# Extrahiere die Anzahl der Studierenden pro Studiengang
studiengang_verteilung = df_2024[df_2024["Category"] == "Anzahl Studierende"][studiengaenge].T

studiengang_verteilung

,66
Bachelor Agrarwissenschaften,293
Bachelor Ernährungswissenschaften,493.0
Bachelor Nachwachsende Rohstoffe und Bioressourcen,69.0
Bachelor Ökotrophologie,429.0
Bachelor Umwelt und globaler Wandel,116.0
Master Agrar- und Ressourcenökonomie,53.0
Master Agrobiotechnologie,139.0
Master Ernährungswissenschaften,338.0
Master Getränketechnologie*,26.0
Master Informationstechnologie in den Agrar- und Umweltwissenschaften*,13.0


In [35]:
studiengang_verteilung.columns = ["Anzahl Studierende"]
studiengang_verteilung["Studiengang"] = studiengang_verteilung.index
studiengang_verteilung.index
studiengang_verteilung

,Anzahl Studierende,Studiengang
Bachelor Agrarwissenschaften,293,Bachelor Agrarwissenschaften
Bachelor Ernährungswissenschaften,493.0,Bachelor Ernährungswissenschaften
Bachelor Nachwachsende Rohstoffe und Bioressourcen,69.0,Bachelor Nachwachsende Rohstoffe und Bioressou...
Bachelor Ökotrophologie,429.0,Bachelor Ökotrophologie
Bachelor Umwelt und globaler Wandel,116.0,Bachelor Umwelt und globaler Wandel
Master Agrar- und Ressourcenökonomie,53.0,Master Agrar- und Ressourcenökonomie
Master Agrobiotechnologie,139.0,Master Agrobiotechnologie
Master Ernährungswissenschaften,338.0,Master Ernährungswissenschaften
Master Getränketechnologie*,26.0,Master Getränketechnologie*
Master Informationstechnologie in den Agrar- und Umweltwissenschaften*,13.0,Master Informationstechnologie in den Agrar- u...


In [37]:
# Treemap erstellen
fig = px.treemap(
    studiengang_verteilung,
    path=["Studiengang"],
    values="Anzahl Studierende",
    title="Studierendenverteilung nach Studiengang (2024)",
    color_discrete_sequence=px.colors.qualitative.Safe,
    template="infoviz"
)

fig.show()

In [94]:
import plotly.express as px


df_2024 = cleaned_sheets["2024"]

# Studiengänge automatisch erkennen (Bachelor und Master)
bachelor_studiengaenge = [col for col in df_2024.columns if "Bachelor" in col]
master_studiengaenge = [col for col in df_2024.columns if "Master" in col]

# Extrahiere die Anzahl der Studierenden pro Studiengang
studiengang_verteilung = df_2024[df_2024["Category"] == "Anzahl Studierende"][bachelor_studiengaenge + master_studiengaenge].T
studiengang_verteilung.columns = ["Anzahl Studierende"]
studiengang_verteilung["Studiengang"] = studiengang_verteilung.index

# Hinzufügen der übergeordneten Kategorie ("Bachelor" oder "Master")
studiengang_verteilung["Studienabschluss"] = studiengang_verteilung["Studiengang"].apply(
    lambda x: "Bachelor" if "Bachelor" in x else "Master"
)


# Farbkodierung für Studiengänge:  
# Bachelor-Studiengänge erhalten Farben aus "Set1", Master-Studiengänge aus "Set2"
bachelor_colors = px.colors.qualitative.Safe
master_colors = px.colors.qualitative.Safe

# Erstellen einer Farbkodierung für jeden Studiengang
color_map = {}
bachelor_count = 0
master_count = 0

for studiengang in studiengang_verteilung["Studiengang"]:
    if "Bachelor" in studiengang:
        color_map[studiengang] = bachelor_colors[bachelor_count % len(bachelor_colors)]
        bachelor_count += 1
    else:
        color_map[studiengang] = master_colors[master_count % len(master_colors)]
        master_count += 1

# Entferne "Bachelor" und "Master" aus den Studiengangsnamen
studiengang_verteilung["Studiengang"] = studiengang_verteilung["Studiengang"].str.replace("Bachelor ", "", regex=False)
studiengang_verteilung["Studiengang"] = studiengang_verteilung["Studiengang"].str.replace("Master ", "", regex=False)

studiengang_verteilung

,Anzahl Studierende,Studiengang,Studienabschluss
Bachelor Agrarwissenschaften,293,Agrarwissenschaften,Bachelor
Bachelor Ernährungswissenschaften,493.0,Ernährungswissenschaften,Bachelor
Bachelor Nachwachsende Rohstoffe und Bioressourcen,69.0,Nachwachsende Rohstoffe und Bioressourcen,Bachelor
Bachelor Ökotrophologie,429.0,Ökotrophologie,Bachelor
Bachelor Umwelt und globaler Wandel,116.0,Umwelt und globaler Wandel,Bachelor
Master Agrar- und Ressourcenökonomie,53.0,Agrar- und Ressourcenökonomie,Master
Master Agrobiotechnologie,139.0,Agrobiotechnologie,Master
Master Ernährungswissenschaften,338.0,Ernährungswissenschaften,Master
Master Getränketechnologie*,26.0,Getränketechnologie*,Master
Master Informationstechnologie in den Agrar- und Umweltwissenschaften*,13.0,Informationstechnologie in den Agrar- und Umwe...,Master


In [ ]:
fig = px.pie(
    studiengang_verteilung,
    values="Anzahl Studierende",
    title="Studierendenverteilung nach Studiengang und Abschluss (2024)",
    color="Studiengang",  # Jeder Studiengang bekommt eine eigene Farbe
    color_discrete_map=color_map,
    template="infoviz"
)

# Schriftgrößen anpassen
fig.update_layout(
    font=dict(size=18),  # Generelle Schriftgröße anpassen
)

# Schriftgröße für Labels innerhalb der Treemap anpassen
fig.update_traces(
    textfont=dict(size=18, color="black", weight="bold"),  # Größe der Texte innerhalb der Treemap
)
fig.update_layout(
    width=1200,  # Breite in Pixeln
    height=800   # Höhe in Pixeln
)

fig.show()

In [95]:
# Treemap erstellen
fig = px.treemap(
    studiengang_verteilung,
    path=["Studienabschluss", "Studiengang"],  # Erstes Level: Bachelor/Master, Zweites Level: Studiengang
    values="Anzahl Studierende",
    title="Studierendenverteilung nach Studiengang und Abschluss (2024)",
    color="Studiengang",  # Jeder Studiengang bekommt eine eigene Farbe
    color_discrete_map=color_map,
    template="infoviz"
)

# Schriftgrößen anpassen
fig.update_layout(
    font=dict(size=18),  # Generelle Schriftgröße anpassen
)

# Schriftgröße für Labels innerhalb der Treemap anpassen
fig.update_traces(
    textinfo="label+value",  # Zeigt den Namen und die Anzahl an
    textfont=dict(size=18, color="black", weight="bold"),  # Größe der Texte innerhalb der Treemap
)
fig.update_layout(
    width=1200,  # Breite in Pixeln
    height=800   # Höhe in Pixeln
)

fig.show()

In [45]:
# Treemap erstellen
fig = px.bar(
    studiengang_verteilung.sort_values(by="Anzahl Studierende", ascending=False),
    x="Studiengang",
    y="Anzahl Studierende",
    title="Studierendenverteilung nach Studiengang (2024)",
    color_discrete_sequence=px.colors.qualitative.Safe,
    template="infoviz"
)

fig.show()

es gibt viele Master studiengänge, aber die drei größten Bachelorstudiengänge haben xx% der Studierenden
Bachelor eher breit, Master spezialisiert

In [64]:
import pandas as pd
import plotly.express as px

# Lade die Daten für alle verfügbaren Jahre
jahre = ['2013', '2019', '2020', '2021', '2022', '2023', '2024']
dfs = []

for jahr in jahre:
    df_jahr = pd.read_excel('data.xlsx', sheet_name=jahr)
    
    # Absolute Anzahl Studierende auswählen
    abs_anzahl = df_jahr.loc[df_jahr['Category'] == 'Anzahl Studierende', 'Anzahl Studierende'].sum()
    
    dfs.append({'Jahr': jahr, 'Anzahl Studierende': df_jahr})

# DataFrame erstellen
df_abs_studierende = pd.DataFrame({
    "Jahr": jahre,
    "Anzahl Studierende": [df.sum(axis=1, numeric_only=True).sum() for df in dfs]
})

# Plot erstellen
fig = px.bar(
    df_abs_studierende,
    x="Jahr",
    y="Anzahl Studierende",
    title="Absolute Anzahl aller Studierenden über die Jahre hinweg",
    labels={"Anzahl Studierende": "Anzahl Studierende", "Jahr": "Jahr"}
)

fig.update_layout(template='infoviz')
fig.show()

KeyError: 'Anzahl Studierende'

In [120]:
import plotly.express as px

df_2024 = cleaned_sheets["2024"]

# Studiengänge automatisch erkennen (Bachelor und Master)
bachelor_studiengaenge = [col for col in df_2024.columns if "Bachelor" in col]
master_studiengaenge = [col for col in df_2024.columns if "Master" in col]

# Extrahiere die Anzahl der Studierenden pro Studiengang
studiengang_verteilung = df_2024[df_2024["Category"] == "Anzahl Studierende"][bachelor_studiengaenge + master_studiengaenge].T
studiengang_verteilung.columns = ["Anzahl Studierende"]
studiengang_verteilung["Studiengang"] = studiengang_verteilung.index

# **Fix: Konvertiere "Anzahl Studierende" zu Float**
studiengang_verteilung["Anzahl Studierende"] = pd.to_numeric(studiengang_verteilung["Anzahl Studierende"], errors="coerce").fillna(0)

# Die vier größten Studiengänge identifizieren
top4_studiengaenge = studiengang_verteilung.nlargest(4, "Anzahl Studierende")["Studiengang"].tolist()

# Farbzuordnung: Top 4 in festen Farben, Rest in Grau
color_map = {studiengang: px.colors.qualitative.Safe[i] for i, studiengang in enumerate(top4_studiengaenge)}
color_map.update({studiengang: "lightgrey" for studiengang in studiengang_verteilung["Studiengang"] if studiengang not in top4_studiengaenge})

# Pie-Chart erstellen
fig = px.pie(
    studiengang_verteilung,
    values="Anzahl Studierende",
    names="Studiengang",
    title="Die vier gròßten Studiengänge beinhalten üver 50% aller Studierenden",
    color="Studiengang",
    color_discrete_map=color_map,
    template="infoviz",

    
)

# Schriftgrößen anpassen
fig.update_layout(
    font=dict(size=18),  # Generelle Schriftgröße anpassen
    width=1200,  # Breite in Pixeln
    height=800,   # Höhe in Pixeln
    showlegend=False
)

# Labels innerhalb des Pie-Charts anpassen
fig.update_traces(
    textfont=dict(size=18, color="black", weight="bold"),
    textinfo="none",
    marker=dict(line=dict(color="grey", width=1)) 
)

fig.show()